In [1]:
import logging
import requests
import pandas as pd
import math
import numpy as np
import re
import sqlite3
import os
from sqlalchemy import create_engine
from datetime import datetime
from bs4 import BeautifulSoup

#Data Extraction
def data_extraction():
    url = 'https://www2.hm.com/en_us/men/products/jeans.html'
    url_base = 'https://www2.hm.com/en_us/'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    bs = BeautifulSoup(page.text, 'html.parser')
    total_itens = bs.find_all('h2', class_='load-more-heading')[0].get('data-total')
    page_number = math.ceil(int(total_itens)/36)
    new_url = url+'?page-size='+str(int(page_number*36))
    new_page = requests.get(new_url, headers=headers)
    new_bs = BeautifulSoup(new_page.text, 'html.parser')
    #Iteração para adquirir o ID de cada produto exposto na vitrine
    product_id = [x.get('data-articlecode') for x in new_bs.find_all('article', class_='hm-product-item')]
    #Iteração para acessar cada item no catálogo, dentro da página de Jeans Masculino
    p_id = []
    item_id = []
    item_name = []
    item_color = []
    item_price = []
    item_composition1 = []
    item_composition2 = []
    ic1 = 0
    ic2 = 0
    item_scrapetime = []
    c=0
    for cod in product_id:
        logger.info('Product %s', cod)
        c+=1
        print(c,"/",total_itens)
        current_url = url_base+str('productpage.'+cod+'.html')
        #print(current_url)
        current_page = requests.get(current_url, headers=headers)
        current_bs = BeautifulSoup(current_page.text, 'html.parser')
        sublist = current_bs.find_all('div', class_='mini-slider')[0].find_all('li', class_='list-item')
        #Iteração para adicionar os códigos de cada subitem em uma lista
        subcodes = [x.find('a').get('data-articlecode') for x in current_bs.find_all('div', class_='mini-slider')[0].find_all('li', class_='list-item')]
        #Iteração para acessar cada subitem na página de um produto singular
        for subcode in subcodes:
            logger.info('Subitem %s', subcode)
            current_url = url_base+str('productpage.'+subcode+'.html')
            current_page = requests.get(current_url, headers=headers)
            current_bs = BeautifulSoup(current_page.text, 'html.parser')
            p_id.append(cod)
            item_id.append(subcode)
            item_name.append(str(current_bs.title.get_text()).split('-')[0])
            item_color.append(str(current_bs.title.get_text()).split('-')[1])
            item_price.append(current_bs.find('span', class_='price-value').get_text().strip())
            item_scrapetime.append(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            #Iteração para localizar e adicionar a composição de cada subitem em uma lista
            for dd in current_bs.find_all('dd'):
                if str(dd).find('Cotton')>0:
                    if ic1 == 0:
                        ic1 = dd.get_text()
                        continue
                    elif ic2 == 0:
                        ic2 = dd.get_text()
                        continue
                if dd == current_bs.find_all('dd')[-1]:
                    item_composition1.append(ic1)
                    item_composition2.append(ic2)
                    ic1 = 0
                    ic2 = 0
            #print(item_composition)
            #shell = np.nan
    df = pd.DataFrame([p_id, item_id, item_name, item_color, item_price, item_composition1, item_composition2, item_scrapetime]).T
    df.columns = ['product_id','item_id','item_name','item_color','item_price','item_composition1', 'item_composition2','item_scrapetime']
    return df

#Data Treatment
def data_treatment(df):
    #Item composition
    df['item_composition1'] = df['item_composition1'].str.lower()
    df['item_composition2'] = df['item_composition2'].str.lower()
    ##Iterate for df range, searching for 'lining' in first column, and replacing first column with second column, in case of finding (Because we want shell columns).
    for i in range(0,len(df)):
        if df['item_composition1'][i].find('lining')>=0:
            df['item_composition1'][i] = df['item_composition2'][i]
    ##Iterate for df range, searching for 'shell' in second column, and replacing first column with second column, in case of finding.
    for i in range(0,len(df)):
        if df['item_composition2'][i].find('shell')>=0:
            df['item_composition1'][i] = df['item_composition2'][i]
    df['item_composition'] = df['item_composition1'].str.replace('shell: ','')
    df.drop(['item_composition1','item_composition2'], axis=1, inplace=True)
    df_temp = df['item_composition'].str.split(',', expand=True)
    ##Iterate to collect name of each material
    material = []
    for c in df_temp.columns:
        for l in df_temp[c]:
            if pd.isnull(l) == False and l.split()[0] not in material:
                material.append(l.split()[0])
    ##Collect and reoder each material from columns
    df_ref = pd.DataFrame(index = np.arange(len(df_temp[0])), columns = material)
    for m in material:
        df_0 = pd.DataFrame(index = np.arange(len(df_temp[0])), columns = [m])
        for c in range(0, len(df_temp.columns)):
            df_k = pd.DataFrame(df_temp.loc[df_temp[c].str.contains(m, na=True), c])
            df_k.columns = [m]
            df_0 = df_0.combine_first(df_k)
            if c == len(df_temp.columns)-1:
                df_ref[m] = df_0
    for c in df_ref.columns:
        df_ref[c] = df_ref[c].str.replace(f'{c} ','')
        df_ref[c] = df_ref[c].str.replace('%','')
        df_ref[c] = df_ref[c].apply(lambda x: float(x)/100 if pd.notnull(x) else x)
    df_ref = df_ref.fillna(0)
    df = pd.concat([df,df_ref], axis=1)
    df.drop(['item_composition'], axis=1, inplace=True)
    #Other columns
    df['item_name'] = df['item_name'].str.strip().apply(lambda x: x.replace(' ', '_').lower())
    df['item_color'] = df['item_color'].str.strip().apply(lambda x: x.replace(' ', '_').lower())
    df['item_price'] = df['item_price'].apply(lambda x: x.replace('$', '')).astype(float)
    return df

#Load
def load_db(df):
    query_showroom_schema = '\nCREATE TABLE IF NOT EXISTS showroom(\n'+str(df.dtypes.replace('int64', 'INTEGER,').replace('object', 'TEXT,').replace('float64', 'REAL,'))[0:-15]+'\n)'
    conn = sqlite3.connect('database_hm.sqlite')
    conn.execute(query_showroom_schema)
    conn.commit()
    conn = create_engine('sqlite:///database_hm.sqlite', echo=False)
    df.to_sql('showroom', con=conn, if_exists='replace', index=False)
    query = """
    select *
    from showroom"""
    df = pd.read_sql(query, conn)
    return df

if __name__ == '__main__':
    #logging
    path = 'logging/'
    if not os.path.exists(path + 'logs_hm'):
        os.makedirs(path+'logs_hm')
    logging.basicConfig(
        filename = path + 'logs_hm/webscraping_hm.log',
        level = logging.DEBUG,
        format = '%(asctime)s-%(levelname)s-%(name)s-%(message)s',
        datefmt = '%Y-%m-%d %H:%M:%S'
        )
    logger = logging.getLogger('webscraping_hm')

    #parameters
    url= 'https://www2.hm.com/en_us/'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) 	Chrome/50.0.2661.102 Safari/537.36'}
    df = data_extraction()
    logger.info('data_extraction done')
    df = data_treatment(df)
    logger.info('data_treatment done')
    df = load_db(df)
    logger.info('load_db done')

1 / 73
2 / 73
3 / 73
4 / 73
5 / 73
6 / 73
7 / 73
8 / 73
9 / 73
10 / 73
11 / 73
12 / 73
13 / 73
14 / 73
15 / 73
16 / 73
17 / 73
18 / 73
19 / 73
20 / 73
21 / 73
22 / 73
23 / 73
24 / 73
25 / 73
26 / 73
27 / 73
28 / 73
29 / 73
30 / 73
31 / 73
32 / 73
33 / 73
34 / 73
35 / 73
36 / 73
37 / 73
38 / 73
39 / 73
40 / 73
41 / 73
42 / 73
43 / 73
44 / 73
45 / 73
46 / 73
47 / 73
48 / 73
49 / 73
50 / 73
51 / 73
52 / 73
53 / 73
54 / 73
55 / 73
56 / 73
57 / 73
58 / 73
59 / 73
60 / 73
61 / 73
62 / 73
63 / 73
64 / 73
65 / 73
66 / 73
67 / 73
68 / 73
69 / 73
70 / 73
71 / 73
72 / 73
73 / 73


In [2]:
df

,product_id,item_id,item_name,item_color,item_price,item_scrapetime,cotton,spandex,polyester,elastomultiester
0,0690449022,0690449001,skinny_jeans,light_denim_blue/trashed,13.99,2022-07-19 14:05:16,0.99,0.01,0.0,0.0
1,0690449022,0690449002,skinny_jeans,denim_blue,14.99,2022-07-19 14:05:19,0.98,0.02,0.0,0.0
2,0690449022,0690449006,skinny_jeans,black/washed,7.99,2022-07-19 14:05:22,0.98,0.02,0.0,0.0
3,0690449022,0690449007,skinny_jeans,light_denim_blue,14.99,2022-07-19 14:05:25,0.98,0.02,0.0,0.0
4,0690449022,0690449009,skinny_jeans,black_washed_out,17.99,2022-07-19 14:05:27,0.99,0.01,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
592,0875105003,0875105018,relaxed_jeans,pale_denim_blue,29.99,2022-07-19 14:12:08,1.00,0.00,0.0,0.0
593,0875105003,0875105023,relaxed_jeans,black,29.99,2022-07-19 14:12:08,1.00,0.00,0.0,0.0
594,0875105003,0875105024,relaxed_jeans,light_denim_blue,29.99,2022-07-19 14:12:09,1.00,0.00,0.0,0.0
595,0875105003,0875105026,relaxed_jeans,light_gray,29.99,2022-07-19 14:12:09,1.00,0.00,0.0,0.0
